In [1]:
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_get_app_dir

import os
import json
import _jsonnet

import ipywidgets as widgets

from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual
from ipywidgets import IntText
from IPython.display import display
from IPython.core.display import display, Javascript, HTML
from traitlets import Unicode, Bool, validate, TraitError
from ipywidgets import DOMWidget, register

In [57]:
cv_schema_dir = os.path.join(k12ai_get_app_dir('cv'), 'templates', 'schema')
cv_json = _jsonnet.evaluate_file(os.path.join(cv_schema_dir, 'k12cv.jsonnet'))
k12ai_print(cv_json)

{
    "data": {
        "dataset": {},
        "loader": {
            "test": {},
            "train": {
                "aug_trans": {
                    "shuffle_trans_seq": {},
                    "trans_seq": {}
                },
                "data_trans": {}
            },
            "val": {}
        },
        "name": {
            "cn": "data",
            "en": "data"
        },
        "objs": [
            "dataset",
            "loader"
        ],
        "type": "tab"
    },
    "description": "k12cv configure test\n",
    "hypes": {
        "iterator": {
            "display_iter": {
                "_id_": "solver.display_iter",
                "default": 200,
                "name": {
                    "cn": "Display Iter",
                    "en": "Display Iter"
                },
                "type": "int"
            },
            "max_iters": {
                "_id_": "solver.max_iters",
                "default": 20000,
                "name": {
     

In [58]:
k12cv_conf = json.loads(cv_json)

In [79]:
from ipywidgets import IntText, FloatText, BoundedIntText, BoundedFloatText, Box, HBox, VBox, Tab, Accordion

In [96]:
lan = 'en'
box_layout = Layout(display='flex',
            flex_flow='row wrap',
            align_items='stretch',
            justify_content='flex-start',
            width='100%')
lo = Layout(display='inline-flex', width='100%')
def _parse_config(widget, config):
    __id_ = config.get('_id_', None)
    _name = config.get('name', None)
    _type = config.get('type', None)
    _objs = config.get('objs', None)
    if _type == 'page':
        nxtwdt = Tab(layout=Layout(width='100%'))
        widget.children = list(widget.children) + [nxtwdt]
    elif _type == 'tab':
        widget.set_title(len(widget.children), _name[lan])
        nxtwdt = VBox(layout = box_layout)
        widget.children = list(widget.children) + [nxtwdt] 
    elif _type == 'accordion':
        for wdt in widget.children:
            if isinstance(wdt, Accordion):
                accord = wdt
                break
        else:
            accord = Accordion(layout=lo)
            widget.children = list(widget.children) + [accord]
        accord.set_title(len(accord.children), _name[lan])
        nxtwdt = VBox(
            children = (HBox(layout = lo),  # basic type
                        HBox(layout = lo)), # enum-trigger type
            layout = box_layout)
        accord.children = list(accord.children) + [nxtwdt]
    elif _type == 'object':
        nxtwdt = widget
    elif _type == 'float':  # leaf node
        default = config.get('default', 0.0)
        wdg = BoundedFloatText(
            description = _name[lan],
            value = default,
            min = 0.0,
            max = 100.0,
            step = 0.1,
        )
        basic_hbox = widget.children[0]
        basic_hbox.children = list(basic_hbox.children) + [wdg]
        return widget
    elif _type == 'string-enum': # leaf node
        default = config.get('default', 'None')
        options = []
        if _objs and len(_objs) > 1:
            basic_hbox = widget.children[0]
            for obj in _objs:
                options.append((obj['name'][lan], obj['value']))
            wdg = widgets.Dropdown(
                options = options,
                value = default,
                description = _name[lan])
            basic_hbox.children = list(basic_hbox.children) + [wdg]
        return widget
    elif _type == 'bool-enum-trigger':
        default = config.get('default', False)
        options = []
        if _objs and len(_objs) > 1:
            items = []
            for obj in _objs:
                options.append((obj['name'][lan], obj['value']))
                if obj.get('trigger', None):
                    tvbox = VBox(layout = lo)
                    _parse_config(tvbox, obj['trigger'])
                    items.append(tvbox)
                    
            wdg = widgets.Dropdown(
                options = options,
                value = default,
                description = _name[lan])
            wdg.items = items # save the sub item for trigger
            
            enum_vbox = VBox(children = [wdg], layout = lo)
            enum_hbox = widget.children[1]
            enum_hbox.children = list(enum_hbox.children) + [enum_vbox]
        return widget
    else:
        return 
    
    # parse subobject
    if _objs and len(_objs) > 0:
        for key in _objs:
            val = config.get(key, None)
            if val:
                _parse_config(nxtwdt, val)


page = Box()
_parse_config(page, k12cv_conf)
page

IndexError: tuple index out of range